The more complex version of this would use this pygithub package as shown in here:
- https://python.langchain.com/docs/integrations/toolkits/github

But let's try a simpler version that just sets up a bunch of python functions to automate different github actions like add, commit and pull requests.

Let's start with a simple commit to a branch of some repository.

First, I'll create a github repository using some simple commands.

In [ ]:
# git init -b main

# git add . && git commit -m "Some commit"

# gh repo create

Now, let's try asking an agent to write come code and commit the resulting code to the current branch of this repository.

To do that, let's give the agent the necessary tools it will need which in this case will be python functions that perform different github actions using the `subprocess` package.

We'll follow the basic steps for building a langchain agent:

# Steps for building a simple agent:
- Set up the LLM
- Define the tool or toolkit (list of tools)
- Set up a prompt template
- Connect llm with your tools
- Define your agent as a dict with keys: input, and agent scratchpad 
- Use the Langchain LCEL language to pipe agent, prompt the llm_with_tools variable and the output parser (which can use OpenAI function calling parser)
- Create the agent loop
- Wrap everything into the AgentExecutor
- Invoke the agent with some query input


In [1]:
# setup the llm
from langchain_openai.chat_models import ChatOpenAI

llm_chat = ChatOpenAI(temperature=0)

Now, let's create a tool for the LLM

In [2]:
# !pip install PyGithub

In [3]:
import subprocess

def github_commit_tool(commit_message="Some commit"):
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", commit_message])
    subprocess.run(["git", "push", "-u", "origin", "main"])
    
    return "Committed to Github"

Now! Before we use it with langchain, let's test it by itself.

In [6]:
# github_commit_tool("Some testing commit")

Now, let's take a look at out github repository from the terminal.

![](2023-10-29-20-47-54.png)

Nice! It looks like we are good to go with this first simple tool!

Let's now make it a tool for our langchain agent by adding the @tool. 

In [15]:
from langchain.tools import tool

@tool
def github_commit_tool(commit_message="Some commit"):
    """This function uses the subprocess package to make commits to a github repo pre-defined."""
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", commit_message])
    subprocess.run(["git", "push", "-u", "origin", "main"])
    
    return "Committed to Github"

tools = [github_commit_tool]

We added some documentation to our function to abide by the requirements of the tool decorator from langchain.

Now, let's test if a simple agent can use that tool!

Let's start by setting up our prompt template.

In [13]:
from langchain.agents import AgentExecutor
from langchain.prompts import ChatPromptTemplate

template = """
You are a helpful programming assistant. The user will ask you to perform commits to a github repo and you will perform the needed actions and call the necessary
functions when needed.
Your current task:
{input}
"""
prompt = ChatPromptTemplate.from_template(template)

In [22]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [23]:
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI

llm_chat = ChatOpenAI(temperature=0)
agent = create_openai_functions_agent(llm_chat, tools, prompt)

In [24]:
from langchain.agents import AgentExecutor


agent_executor = AgentExecutor(agent=agent,tools=tools, verbose=True)

In [27]:
agent_executor.invoke({"input": "Create a github commit with the message 'Some test commit'"})



> Entering new AgentExecutor chain...

Invoking: `github_commit_tool` with `{'commit_message': 'Some test commit'}`


[main 58cf923] Some test commit
 1 file changed, 138 insertions(+), 71 deletions(-)


To https://github.com/EnkrateiaLucca/oreilly_live_training_agents.git
 ! [rejected]        main -> main (non-fast-forward)
error: failed to push some refs to 'https://github.com/EnkrateiaLucca/oreilly_live_training_agents.git'
hint: Updates were rejected because the tip of your current branch is behind
hint: its remote counterpart. Integrate the remote changes (e.g.
hint: 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


Committed to GithubA commit with the message 'Some test commit' has been created on Github.

> Finished chain.


{'input': "Create a github commit with the message 'Some test commit'",
 'output': "A commit with the message 'Some test commit' has been created on Github."}

And voila, the agent calls the right function and creates the commit!

(ignore the github related error, let's focus on the agent capabilities).

And voila! We made a commit!

![](2023-10-30-13-04-20.png)

Ok, this is cool but, we can go beyond that, and add the ability to create a file and then commit that file to our desired repository. Let's do that!

To do it all we have to do is amplify the Agent's toolkit with another tool that can create files in the current folder.

Perfect! Now, let's add this to our github agent

In [28]:
@tool
def create_file_tool(file_contents):
    """This function creates a file with the contents and file path provided as inputs."""
    with open("file.py", "w") as f:
        f.write(file_contents)
    
    return "File created"

Now, let's add that functionality to our agent by just updating the tools list.

In [29]:
tools = [github_commit_tool, create_file_tool]

Now, let's modify our agent with its new toolkit

In [30]:
agent = create_openai_functions_agent(llm_chat, tools, prompt)

In [31]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Now, let's try using both tools in a call to see if the agent can do something slightly more complex now.

In [32]:
agent_executor.invoke({
    "input": "Write Python code for summing two numbers. Output ONLY THE CODE and save that to a file then commit it to our github repository."
})



> Entering new AgentExecutor chain...

Invoking: `create_file_tool` with `{'file_contents': 'def sum_numbers(a, b):\n    return a + b'}`


File created
Invoking: `github_commit_tool` with `{'commit_message': 'Add code for summing two numbers'}`


[main fda9a37] Add code for summing two numbers
 1 file changed, 2 insertions(+)
 create mode 100644 notebooks/file.py


To https://github.com/EnkrateiaLucca/oreilly_live_training_agents.git
   ef22ee1..fda9a37  main -> main


branch 'main' set up to track 'origin/main'.
Committed to GithubThe Python code for summing two numbers has been saved to a file and committed to your GitHub repository.

> Finished chain.


{'input': 'Write Python code for summing two numbers. Output ONLY THE CODE and save that to a file then commit it to our github repository.',
 'output': 'The Python code for summing two numbers has been saved to a file and committed to your GitHub repository.'}

And voila again!
![](2023-10-30-13-15-38.png)

For The Practice Demo We'll stop here, below I show how to expand this for multi-input tools.

Ok! This is awesome right? Let's just stop for a second and appreciate the simplicity of this implementation and the power that it gives us! Now, on the other hand, we are using a [ZeroShotReactAgent](https://python.langchain.com/docs/modules/agents/agent_types/react#:~:text=Using-,zeroshotreactagent), which is limited when it comes to multi input functions as tools. However, we can easily expand this by leveraging Langchain's amazing [Multi-Input Tools](https://python.langchain.com/docs/modules/agents/tools/multi_input_tool#:~:text=On%20this%20page-,Multi-Input%20Tools,with%20the%20StructuredTool%20class.,-import%20os).

Let's do that to modify our current tool: `create_file_tool()` to accept the file's contents as well as a file path, this way we are not restricted in creating just one type of file with a pre-specified name.

In [33]:
from langchain.tools import StructuredTool

def create_file_tool(file_contents: str, file_path: str) -> str:
    """
    Function that creates a file with the contents 
    and file path provided as inputs.
    """
    with open(file_path, "w") as f:
        f.write(file_contents)
    
    return "File created"

# Let's test it first!
create_file_tool("print('Hello World')", "print_file.py")

'File created'

In [34]:
!ls *.py

file.py                            print_file.py
function_calls_chatgpt.py          sample_app_to_test_github_agent.py


In [35]:
!python print_file.py

Hello World


Great! Now, let's make it a tool for the agent.

In [36]:
create_file_tool = StructuredTool.from_function(create_file_tool)

And, again, let's update our toolkit and our agent.

In [37]:
tools = [github_commit_tool, create_file_tool]

In [38]:
agent = create_openai_functions_agent(llm_chat, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Now, let's test it by asking it to write a simple `README.md` file, giving the agent some context about the project's repo.

In [40]:
agent_executor.invoke({
    "input": "Write file named testing-github-agent.md file with a sample toy example tutorial."
})



> Entering new AgentExecutor chain...

Invoking: `create_file_tool` with `{'file_contents': '## Toy Example Tutorial\n\nThis is a sample tutorial for testing the GitHub agent.\n\n1. Step 1: Clone the repository\n\n   ```\n   git clone https://github.com/example-repo.git\n   ```\n\n2. Step 2: Create a new branch\n\n   ```\n   git checkout -b new-branch\n   ```\n\n3. Step 3: Make changes\n\n   ```\n   // Make your changes here\n   ```\n\n4. Step 4: Commit and push\n\n   ```\n   git add .\n   git commit -m "Added new feature"\n   git push origin new-branch\n   ```\n\n5. Step 5: Create a pull request\n\n   ```\n   // Go to the repository on GitHub\n   // Click on the "New pull request" button\n   // Select the new-branch as the base branch\n   // Add a title and description for the pull request\n   // Click on the "Create pull request" button\n   ```\n\n6. Step 6: Review and merge\n\n   ```\n   // Wait for the pull request to be reviewed\n   // If approved, click on the "Merge pull reque

{'input': 'Write file named testing-github-agent.md file with a sample toy example tutorial.',
 'output': 'I have created a file named "testing-github-agent.md" with a sample toy example tutorial.'}

And again we succeed!

![](2023-10-30-13-26-32.png)

Ok, before we finish with this example, let's add a last tool to read files from a repo, for situations where we might write some Python code and we want our agent to integrate that Python code into our repository. 

Again, we follow the same procedure:

- Create the tool
- Update our tookit
- Update our agent
- Run a test task to check if it works 

In [41]:
def read_file_tool(file_path):
    with open(file_path, "r") as f:
        contents = f.read()
    
    return contents

read_file_tool("testing-github-agent.md")

'## Toy Example Tutorial\n\nThis is a sample tutorial for testing the GitHub agent.\n\n1. Step 1: Clone the repository\n\n   ```\n   git clone https://github.com/example-repo.git\n   ```\n\n2. Step 2: Create a new branch\n\n   ```\n   git checkout -b new-branch\n   ```\n\n3. Step 3: Make changes\n\n   ```\n   // Make your changes here\n   ```\n\n4. Step 4: Commit and push\n\n   ```\n   git add .\n   git commit -m "Added new feature"\n   git push origin new-branch\n   ```\n\n5. Step 5: Create a pull request\n\n   ```\n   // Go to the repository on GitHub\n   // Click on the "New pull request" button\n   // Select the new-branch as the base branch\n   // Add a title and description for the pull request\n   // Click on the "Create pull request" button\n   ```\n\n6. Step 6: Review and merge\n\n   ```\n   // Wait for the pull request to be reviewed\n   // If approved, click on the "Merge pull request" button\n   // Confirm the merge\n   ```\n\nThat\'s it! You have successfully completed the

Ok, our function seems to work so let's add it to our toolkit, since its a simple function with only one input we can just add the @tool decorator on top of it.

In [42]:
@tool
def read_file_tool(file_path):
    """
    Simple function to read contents from a file.
    """
    with open(file_path, "r") as f:
        contents = f.read()
    
    return contents

And we update our toolkit:

In [43]:
tools = [github_commit_tool, create_file_tool, read_file_tool]

And we update our agent

In [44]:
agent = create_openai_functions_agent(llm_chat, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Before we run our newly updated agent, let's create a .txt file containing the contents from this notebook so that our agent has something useful to read.

In [53]:
agent_executor.invoke(
    {
    "input": "Write a file named summary_tutorial.md containing a sentence summary of the contents in the ./testing-github-agent.md file and make a commit."
    }
                      )



> Entering new AgentExecutor chain...

Invoking: `read_file_tool` with `{'file_path': './testing-github-agent.md'}`


## Toy Example Tutorial

This is a sample tutorial for testing the GitHub agent.

1. Step 1: Clone the repository

   ```
   git clone https://github.com/example-repo.git
   ```

2. Step 2: Create a new branch

   ```
   git checkout -b new-branch
   ```

3. Step 3: Make changes

   ```
   // Make your changes here
   ```

4. Step 4: Commit and push

   ```
   git add .
   git commit -m "Added new feature"
   git push origin new-branch
   ```

5. Step 5: Create a pull request

   ```
   // Go to the repository on GitHub
   // Click on the "New pull request" button
   // Select the new-branch as the base branch
   // Add a title and description for the pull request
   // Click on the "Create pull request" button
   ```

6. Step 6: Review and merge

   ```
   // Wait for the pull request to be reviewed
   // If approved, click on the "Merge pull request" button
   // Co

To https://github.com/EnkrateiaLucca/oreilly_live_training_agents.git
   fda9a37..d3fc425  main -> main


branch 'main' set up to track 'origin/main'.
Committed to GithubI have created a file named `summary_tutorial.md` containing a sentence summary of the contents in the `./testing-github-agent.md` file. I have also made a commit with the message "Add summary tutorial file".

> Finished chain.


{'input': 'Write a file named summary_tutorial.md containing a sentence summary of the contents in the ./testing-github-agent.md file and make a commit.',
 'output': 'I have created a file named `summary_tutorial.md` containing a sentence summary of the contents in the `./testing-github-agent.md` file. I have also made a commit with the message "Add summary tutorial file".'}

In [55]:
!cat summary_tutorial.md

# Summary of Tutorial

This tutorial provides a step-by-step guide on how to use the GitHub agent to clone a repository, create a new branch, make changes, commit and push the changes, create a pull request, and review and merge the pull request.

![](2023-10-30-13-42-25.png)

I think this is a great usecase for agents like these even if sometimes the performance and reliability is a bit off, because now, its a matter of optimizing the edges to get it to work for complex use cases.